In [ ]:
#@title 無論如何先點擊左邊「開始」按鍵
!pip install pikepdf
!pip install reportlab
!pip install PyPDF4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 4.2 MB/s 
     |████████████████████████████████| 3.2 MB 29.9 MB/s 
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.8 MB 4.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Created wheel for PyPDF4: filename=PyPDF4-1.27.0-py3-none-any.whl size=61248 sha256=a6b1045049e71eb323279e04128a850bb5ab9b574ba49cc32fc73998fbee9167
  Stored in directory: /root/.cache/pip/wheels/f0/79/75/d130281ec9996a2551dbdd1836aa4beb376d53f8cdca49b4b0
Successfully built PyPDF4


#**PDF分割**

In [ ]:
#@title 填寫完下方資料後，點擊左邊「開始」按鍵
#@markdown * 把檔案拉到左色選單>點擊檔案>右鍵>複製路徑>貼到下方
File_Path = '/content/doc02158220220708104502.pdf' #@param {type:"string"}
#@markdown * 起始頁數(自行調整)
Start_Page = 1 #@param {type:"integer"}
#@markdown * 結束頁數(自行調整)
End_Page = 2 #@param {type:"integer"}
#@markdown * 新檔案名稱
New_Spilt_File_Name = '123' #@param {type:"string"}

from pikepdf import Pdf

pdf = Pdf.open(File_Path)   
pages = pdf.pages

output = Pdf.new()
if End_Page <= len(pages):
    for i in range(Start_Page - 1, End_Page):
        output.pages.append(pages[i])
    output.save('/content/' + New_Spilt_File_Name + '.pdf')
else:
    print('超過原本頁數')

#**PDF合併**

In [ ]:
#@title 填寫完下方資料後，點擊左邊「開始」按鍵
#@markdown * 把「檔案1」拉到左色選單>點擊檔案>右鍵>複製路徑>貼到下方
File_Path_1 = '/content/\u7C3D.pdf' #@param {type:"string"}
#@markdown * 把「檔案2」拉到左色選單>點擊檔案>右鍵>複製路徑>貼到下方
File_Path_2 = '/content/\u7A3F.pdf' #@param {type:"string"}
#@markdown * 新檔案名稱
New_Merge_File_Name = '\u7C3D\u7A3F\u5408\u4F75' #@param {type:"string"}
from pikepdf import Pdf
pdf_1 = Pdf.open(File_Path_1)
pdf_2 = Pdf.open(File_Path_2)
pages_1 = pdf_1.pages
pages_2 = pdf_2.pages
#pages = pdf.pages
output = Pdf.new()  
for i in range(0, len(pages_1)):
  output.pages.append(pages_1[i])  
for j in range(0, len(pages_2)):
  output.pages.append(pages_2[j])
output.save('/content/' + New_Merge_File_Name + '.pdf')

#**PDF編寫頁碼**

In [ ]:
#@title 填寫完下方資料後，點擊左邊「開始」按鍵
#@markdown * 把檔案拉到左色選單>點擊檔案>右鍵>複製路徑>貼到下方
#@markdown * 檔案新名稱是「原檔名_num.pdf」
File_Path_4_Page_Num = '/content/hw3_num.pdf' #@param {type:"string"}
#@markdown * 頁碼大小
Font_Size = 32 #@param {type:"integer"}
#@markdown * 頁碼位置
Font_Place = "\u4E2D" #@param ["中", "右"]

import os
from PyPDF4.pdf import PdfFileReader, PdfFileWriter
from reportlab.lib.units import mm
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter

def Create_Page_Pdf(num, tmp):
    c = canvas.Canvas(tmp)
    for i in range(1, num + 1):
        
        # 判斷頁面版面方向
        pdf = PdfFileReader(File_Path_4_Page_Num)
        size = pdf.getPage(i - 1).mediaBox
        width, height = size[2], size[3]
        
        if Font_Place == "\u4E2D": # 中
          if width > height: # 橫向
            # 將初始座標(20, 340)設為新座標(0,0)，並旋轉270度
            c.translate(20, 340)
            c.rotate(270)
          else:
            # 將初始座標(285, 20)設為新座標(0,0)
            c.translate(285, 20)
        elif Font_Place == "\u53F3": # 右
          if width > height: # 縱向
            # 將初始座標(20, 40)設為新座標(0,0)，並旋轉270度
            c.translate(20, 40)
            c.rotate(270)
          else:
            # 將初始座標(550, 20)設為新座標(0,0)
            c.translate(550, 20)
        
        # Helvetica是一種廣泛使用於拉丁字母的無襯線字體
        c.setFont("Helvetica", Font_Size)
        c.drawString(mm, -mm, str(i)) 
        c.showPage()
    c.save()

def Add_Page_Numbers(pdf_path):
    tmp = "__tmp.pdf"

    writer = PdfFileWriter()
    with open(File_Path_4_Page_Num, "rb") as f:
        reader = PdfFileReader(f, strict=False)
        # 獲得總頁數
        File_Pages = reader.getNumPages()

        Create_Page_Pdf(File_Pages, tmp)

        with open(tmp, "rb") as ftmp:
            number_pdf = PdfFileReader(ftmp)
            for p in range(File_Pages):
                page = reader.getPage(p)
                numberLayer = number_pdf.getPage(p)
                page.mergePage(numberLayer)
                writer.addPage(page)
            if writer.getNumPages():
                newpath = pdf_path[:-4] + "_num.pdf"
                with open(newpath, "wb") as f:
                    writer.write(f)
        os.remove(tmp)
Add_Page_Numbers(File_Path_4_Page_Num)

In [ ]:
#@title 填寫完下方資料後，點擊左邊「開始」按鍵
#@markdown * 把檔案拉到左色選單>點擊檔案>右鍵>複製路徑>貼到下方
#@markdown * 檔案新名稱是「原檔名_num.pdf」
File_Path_4_Page_Num = '/content/hw3_num.pdf' #@param {type:"string"}
#@markdown * 頁碼大小
Font_Size = 32 #@param {type:"integer"}
#@markdown * 頁碼位置
Font_Place = "\u4E2D" #@param ["中", "右"]

import os
from PyPDF4.pdf import PdfFileReader, PdfFileWriter
from reportlab.lib.units import mm
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter

def Create_Page_Pdf(num, tmp):
    c = canvas.Canvas(tmp)
    for i in range(1, num + 1):
        
        # 判斷頁面版面方向
        pdf = PdfFileReader(File_Path_4_Page_Num)
        size = pdf.getPage(i - 1).mediaBox
        width, height = size[2], size[3]
        
        if Font_Place == "\u4E2D": # 中
          if width > height: # 橫向
            # 將初始座標(20, 340)設為新座標(0,0)，並旋轉270度
            c.translate(20, 340)
            c.rotate(270)
          else:
            # 將初始座標(285, 20)設為新座標(0,0)
            c.translate(285, 20)
        elif Font_Place == "\u53F3": # 右
          if width > height: # 縱向
            # 將初始座標(20, 40)設為新座標(0,0)，並旋轉270度
            c.translate(20, 40)
            c.rotate(270)
          else:
            # 將初始座標(550, 20)設為新座標(0,0)
            c.translate(550, 20)
        
        # Helvetica是一種廣泛使用於拉丁字母的無襯線字體
        c.setFont("Helvetica", Font_Size)
        c.drawString(mm, -mm, str(i)) 
        c.showPage()
    c.save()

def Add_Page_Numbers(pdf_path):
    tmp = "__tmp.pdf"

    writer = PdfFileWriter()
    with open(File_Path_4_Page_Num, "rb") as f:
        reader = PdfFileReader(f, strict = False)
        # 獲得總頁數
        File_Pages = reader.getNumPages()

        Create_Page_Pdf(File_Pages, tmp)

        with open(tmp, "rb") as ftmp:
            number_pdf = PdfFileReader(ftmp)
            for p in range(File_Pages):
                page = reader.getPage(p)
                numberLayer = number_pdf.getPage(p)
                page.mergePage(numberLayer)
                writer.addPage(page)
            if writer.getNumPages():
                newpath = pdf_path[:-4] + "_num.pdf"
                with open(newpath, "wb") as f:
                    writer.write(f)
        os.remove(tmp)
Add_Page_Numbers(File_Path_4_Page_Num)